# Импорт библиотек

In [1]:
import pandas as pd 

import os
import logging

# Загрузка данных

In [2]:
df = pd.read_csv('data/KaggleV2-May-2016.csv')

# Логирование информации о данных

✍️ Теперь, когда вы понимаете основы модуля logging в Python, посмотрим, как можно его использовать при проведении экспериментов по машинному обучению. Давайте залогируем информацию о нашем датасете Medical Appointment No Shows. 

Создадим новый ipynb файл, в котором будем выполнять логирование. Скачать оригинальный ноутбук вы можете по ссылке.

Создадим функцию log() для генерации лог-файла и записи в него информации.

In [3]:
# Функция для создания лог-файла и записи в него информации
def get_logger(path, file):
   """[Создает лог-файл для логирования в него]
   Аргументы:
       path {string} -- путь к директории
       file {string} -- имя файла
  
   Возвращает:
       [obj] -- [логер]
   """
 
   # проверяем, существует ли файл
   log_file = os.path.join(path, file)
 
   if not os.path.isfile(log_file):
       open(log_file, "w+").close()
 
   # поменяем формат логирования
   file_logging_format = "%(levelname)s: %(asctime)s: %(message)s"
 
   # конфигурируем лог-файл
   logging.basicConfig(level=logging.INFO, 
   format = file_logging_format)
   logger = logging.getLogger()
  
   # создадим хэнлдер для записи лога в файл
   handler = logging.FileHandler(log_file)
 
   # установим уровень логирования
   handler.setLevel(logging.INFO)
  
   # создадим формат логирования, используя file_logging_format
   formatter = logging.Formatter(file_logging_format)
   handler.setFormatter(formatter)
 
   # добавим хэндлер лог-файлу
   logger.addHandler(handler)
 
   return logger

Итак, get_logger принимает на вход папку, в которой вы хотите хранить файл лога и имя файла. Функция нацелена на создание информационных сообщений и записи их в лог-файл. Для того, чтобы освежить в памяти знания по работе с файлами, вы можете вернуться к Блоку 2 нашего курса («Подгрузка данных»).  

Для удобства лог-файлы хранят в отдельной директории. Новую папку можно создать с помощью команды mkdir:

In [4]:
# создаем папку для логирования
!mkdir logs

mkdir: cannot create directory ‘logs’: File exists


Создадим лог-файл и запишем туда информацию о датасете.

In [5]:
# создаем лог-файл
logger = get_logger(path="logs/", file="data.logs")

In [6]:
logger.info("Data")

INFO: 2023-01-19 13:02:13,758: Data


In [7]:
logger.info("Data shape {}".format(df.shape))

INFO: 2023-01-19 13:02:14,143: Data shape (110527, 14)


In [8]:
logger.info("Percentage of women: {}".format(df[df['No-show']=='Yes']['Gender'].value_counts(True)[0]))

INFO: 2023-01-19 13:02:14,653: Percentage of women: 0.653882342398853


In [9]:
logger.info("Percentage of men: {}".format(df[df['No-show']=='Yes']['Gender'].value_counts(True)[1]))

INFO: 2023-01-19 13:02:15,092: Percentage of men: 0.346117657601147


⭐️ Отлично! Мы зафиксировали информацию о датасете в лог-файл. Такой файл отражает сведения о том, что всё работает в штатном режиме (то есть просто ежедневный мониторинг программы) или что произошла какая-то ошибка, на которую нужно максимально срочно отреагировать и устранить. Например, мы можем проверять форму данных, и, если форма не соответствует необходимой, логировать ошибку:

In [10]:
if df.shape != (110526,14):
  logger.error('Data shape is incorrect!')

ERROR: 2023-01-19 13:02:15,486: Data shape is incorrect!


# 4. Знакомство с Comet.ml

→ Comet.ml — это онлайн-платформа, позволяющая отслеживать эксперименты. Основное преимущество Comet состоит в том, что с её помощью можно легко построить панель отчётности и систему мониторинга.
	

Comet предоставляет следующие возможности:

    сравнивать эксперименты с точки зрения метрик, параметров и так далее;
    следить за моделью от создания до вывода в продакшен;
    делиться своим проектом с другими людьми, которые в режиме реального времени будут следить за результатами;
    строить отчёты исходя из результатов эксперимента;
    оставить проект приватным или сделать его общедоступным.

Для начала работы с платформой Comet.ml необходимо зарегистрировать бесплатную учётную запись. После этого вы сможете создавать как публичные, так и приватные проекты. 

In [11]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="vOutzrCeUclAaITir3N0p7clU",
    project_name="medical-appointment",
    workspace="lurtz-dahaka",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/lurtz-dahaka/medical-appointment/38e484a8ea004b95b755b493b150c6ab



Класс Experiment — это интерфейс локального кода для Comet. Он определяет множество методов, описанных в официальной документации Comet. Платформа позволяет хранить информацию о коде, логировать графики, гиперпараметры модели (о них вы узнаете дальше в курсе), метрики. 

Давайте рассмотрим некоторые популярные методы:

    log_metric() и log_metrics() — логируют в эксперименте одну или несколько оценочных метрик, таких как accuracy;
    log_figure() — логирует рисунок;
    display() — создаёт интерактивную среду в Jupyter, показывающую приборную панель Comet как вывод ячейки;
    end() — если эксперимент выполняется в Jupyter, этот метод указывает, что эксперимент завершён.

После завершения эксперимента Comet предоставляет информационную панель, где можно увидеть все залогированные в коде метрики, цифры, параметры и так далее.

Показатели оценки отображаются автоматически. Например, если эксперимент состоит из нескольких шагов, можно легко нарисовать графики, показывающие метрики в зависимости от количества шагов.

Ниже представлен пример графика, иллюстрирующего изменение accuracy на каждом шаге обучения нейронной сети (про нейронные сети вы узнаете далее по курсу). 

Итак, Comet.ml — это платформа для экспериментов с машинным обучением, которую специалисты по данным используют для отслеживания, сравнения и объяснения своих экспериментов по машинному обучению. Она обеспечивает воспроизведение моделей, простое обслуживание рабочего процесса машинного обучения и бесперебойную совместную работу на протяжении всего жизненного цикла проекта.  

Например, вы в команде строите модель оценки спроса на лекарство. Вы можете разделить пайплайн машинного обучения между участниками проекта, при этом каждый участник получит возможность отслеживать все эксперименты каждого этапа в Comet: от логирования визуализаций до подбора параметров обучения модели и оценки её производительности.